In [1]:
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
import torch
from PIL import Image
import numpy as np
import json

In [2]:
# pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
# pipe2 = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1")
pipe2 = StableDiffusionImg2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-2-1")
pipe = pipe.to("cuda")
pipe2 = pipe2.to("cuda")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

c:\Users\Innomer\.conda\envs\gen\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
strength=0.75

In [4]:
# Generate images from multiple angles
object="chair"
prompts = [f"showcase the {object} from the front view", f"showcase the {object} from the left view", f"showcase the {object} from the back view", f"showcase the {object} from the right view"]
images = [] 
prev_image = None
for idx, prompt in enumerate(prompts):
    if idx>0:
        img=Image.open(prev_image).convert("RGB").resize((512,512))
        prompt="Use the image as the reference and "+prompt
        image = pipe2(prompt=prompt, image=img,strength=strength).images[0]
    else:
        image = pipe(prompt).images[0]
    image.save(f"images\\view_{idx}.png")
    prev_image = f"images\\view_{idx}.png"

c:\Users\Innomer\.conda\envs\gen\Lib\site-packages\transformers\models\clip\modeling_clip.py:491: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
# Function to generate synthetic camera extrinsics
# One-time Function.
def create_camera_pose(angle, distance):
    # Create a simple rotation around the Y-axis for varying viewpoints
    theta = np.radians(angle)
    cam_position = np.array([distance * np.sin(theta), 0, distance * np.cos(theta)])
    
    forward = -cam_position / np.linalg.norm(cam_position)
    up = np.array([0, 1, 0])
    right = np.cross(up, forward)
    
    # Rotation and translation matrix (extrinsic)
    extrinsic_matrix = np.identity(4)
    extrinsic_matrix[0:3, 0] = right
    extrinsic_matrix[0:3, 1] = up
    extrinsic_matrix[0:3, 2] = forward
    extrinsic_matrix[0:3, 3] = cam_position
    
    return extrinsic_matrix

angles = [0, 90, 270, 360]
distance = 3.0

camera_angle_x = np.radians(60)

frames = []
for i, angle in enumerate(angles):
    pose = create_camera_pose(angle, distance)
    frames.append({
        "file_path": f"view_{i}.png",
        "transform_matrix": pose.tolist()
    })

camera_data = {
    "camera_angle_x": camera_angle_x,
    "frames": frames
}

with open('images\camera_data.json', 'w') as f:
    json.dump(camera_data, f, indent=4)

<>:39: SyntaxWarning: invalid escape sequence '\c'
<>:39: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Innomer\AppData\Local\Temp\ipykernel_23540\1444583625.py:39: SyntaxWarning: invalid escape sequence '\c'
  with open('images\camera_data.json', 'w') as f:


In [ ]:
%../instant-ngp/instant-ngp.exe ./images/